In [2]:
import pandas as pd
import numpy as np
import requests
import pprint
import sidrapy
import warnings 
warnings.simplefilter("ignore")

# Estabelecer a conexão com a API do SIDRA IBGE

In [116]:
API = "https://apisidra.ibge.gov.br/values/t/1612/n3/43/v/allxp/p/last%2024/c81/2713"

Transformar o arquivo json em um Dataframe

In [117]:
req = requests.get(API)
info = req.json()

In [140]:
df = pd.DataFrame(info)
df.columns = df.iloc[0]
df = df[1:]
df.reset_index(drop=True, inplace=True)

Exluir colunas desnecessárias

In [141]:
df.drop(['Nível Territorial (Código)', 'Nível Territorial',
       'Unidade de Medida (Código)', 'Variável (Código)', 'Ano (Código)', 'Produto das lavouras temporárias (Código)',
       'Unidade de Medida'
       ],
       axis = 1, inplace = True)

Substituir valores não numéricos da coluna Valor e transformá-la em decimal

In [142]:
df.Valor.replace("-", np.nan, inplace = True)
df.Valor.replace("X", np.nan, inplace = True)
df.Valor.replace("...", np.nan, inplace = True)
df.Valor.replace("..", np.nan, inplace = True)

In [143]:
df.Valor = df.Valor.astype(float)

# Seperar os valores em linha e depois fazer o merge em colunas

In [144]:

area_plant = df.loc[df["Variável"] == "Área plantada"]
area_col = df.loc[df["Variável"] == "Área colhida"]
produtiv = df.loc[df["Variável"] == 'Rendimento médio da produção']
prod = df.loc[df["Variável"] == "Quantidade produzida"]
val = df.loc[df["Variável"] == "Valor da produção"]

In [145]:
area_plant.drop("Variável", axis = 1, inplace = True)
area_col.drop("Variável", axis = 1, inplace = True)
produtiv.drop("Variável", axis = 1, inplace = True)
prod.drop("Variável", axis = 1, inplace = True)
val.drop("Variável", axis = 1, inplace = True)

In [146]:
mer = area_plant.merge(area_col, on = ["Ano", 'Unidade da Federação (Código)',
       'Unidade da Federação', 'Produto das lavouras temporárias'], how = "inner")
mer.rename({
    'Valor_x': 'area_plantada(ha)',
    'Valor_y': 'area_colhida(ha)'
}, axis = 1, inplace = True)

Calcular a diferença entre áreas plantadas e colhidas

In [147]:
mer["plantada-colhida"] = mer['area_plantada(ha)'] - mer['area_colhida(ha)']

In [148]:
mer = mer.merge(prod, on = ["Ano", 'Unidade da Federação (Código)',
       'Unidade da Federação', 'Produto das lavouras temporárias'], how = "inner")
mer.rename({"Valor": "producao(t)"}, axis = 1, inplace = True)

In [149]:
mer = mer.merge(produtiv, on = ["Ano", 'Unidade da Federação (Código)',
       'Unidade da Federação', 'Produto das lavouras temporárias'], how = "inner")
mer.rename({"Valor": "produtividade(kg/ha)"}, axis = 1, inplace = True)

In [150]:
mer = mer.merge(val, on = ["Ano", 'Unidade da Federação (Código)',
       'Unidade da Federação', 'Produto das lavouras temporárias'], how = "inner")
mer.rename({"Valor": "valor_mil(R$)"}, axis = 1, inplace = True)

Calcular o valor médio da saca de soja

In [151]:
mer['valor_saca'] = (mer['valor_mil(R$)'] / ((mer['producao(t)'] / 60))).round(2)

In [152]:
mer.columns

Index(['area_plantada(ha)', 'Unidade da Federação (Código)',
       'Unidade da Federação', 'Ano', 'Produto das lavouras temporárias',
       'area_colhida(ha)', 'plantada-colhida', 'producao(t)',
       'produtividade(kg/ha)', 'valor_mil(R$)', 'valor_saca'],
      dtype='object', name=0)

In [153]:
mer = mer[[ 'Ano', 'Unidade da Federação', 'Produto das lavouras temporárias', 'area_plantada(ha)',
       'area_colhida(ha)', 'plantada-colhida', 'producao(t)', 'produtividade(kg/ha)', 'valor_mil(R$)', 'valor_saca']]

In [154]:
mer.to_parquet("soja_IBGE_RS.parquet", index = False)

# Importar as variáveis climátivas